In [1]:
import json
import pandas as pd
import numpy as np
import os

import re

from sqlalchemy import create_engine
import psycopg2

# from config import db_password

import time

In [2]:
#  Add the clean movie function that takes in the argument, "movie".
# 1. Add the clean movie function that takes in the argument, "movie".
# Added from lessons8.3.6 clean movies combine the earlier function with the new
#merge columns function

def clean_movie(movie):
    movie = dict(movie) #create a non-destructive copy
    alt_titles = {}
    # combine alternate titles into one list
    for key in ['Also known as','Arabic','Cantonese','Chinese','French',
                'Hangul','Hebrew','Hepburn','Japanese','Literally',
                'Mandarin','McCune-Reischauer','Original title','Polish',
                'Revised Romanization','Romanized','Russian',
                'Simplified','Traditional','Yiddish']:
        if key in movie:
            alt_titles[key] = movie[key]
            movie.pop(key)
    if len(alt_titles) > 0:
        movie['alt_titles'] = alt_titles

    # merge column names
    def change_column_name(old_name, new_name):
        if old_name in movie:
            movie[new_name] = movie.pop(old_name)
    change_column_name('Adaptation by', 'Writer(s)')
    change_column_name('Country of origin', 'Country')
    change_column_name('Directed by', 'Director')
    change_column_name('Distributed by', 'Distributor')
    change_column_name('Edited by', 'Editor(s)')
    change_column_name('Length', 'Running time')
    change_column_name('Original release', 'Release date')
    change_column_name('Music by', 'Composer(s)')
    change_column_name('Produced by', 'Producer(s)')
    change_column_name('Producer', 'Producer(s)')
    change_column_name('Productioncompanies ', 'Production company(s)')
    change_column_name('Productioncompany ', 'Production company(s)')
    change_column_name('Released', 'Release Date')
    change_column_name('Release Date', 'Release date')
    change_column_name('Screen story by', 'Writer(s)')
    change_column_name('Screenplay by', 'Writer(s)')
    change_column_name('Story by', 'Writer(s)')
    change_column_name('Theme music composer', 'Composer(s)')
    change_column_name('Written by', 'Writer(s)')

    return movie

In [3]:
#delete later: create file path for testing

# 6 Create the path to your file directory and variables for the three files. 
file_dir = os.path.join("../Movies-ETL")
# Wikipedia data
wiki_file = f'{file_dir}/wikipedia_movies.json'
# Kaggle metadata
kaggle_file = f'{file_dir}/movies_metadata.csv'
# MovieLens rating data.
ratings_file = f'{file_dir}/ratings.csv'

In [95]:
# 1 Add the function that takes in three arguments;
# Wikipedia data, Kaggle metadata, and MovieLens rating data (from Kaggle)
#step 2 testing block

def ETL():
    
    # 2. Read in the kaggle metadata and MovieLens ratings CSV files as Pandas DataFrames.
    kaggle_file = pd.read_csv('movies_metadata.csv', low_memory = False)
    ratings_file = pd.read_csv('ratings.csv', low_memory = False)

    # 3. Open the read the Wikipedia data JSON file.
    with open(f'{file_dir}/wikipedia_movies.json', mode='r') as file:
        wiki_movies = json.load(file)
    
     # 3. Write a list comprehension to filter out TV shows.
    wiki_file = [movie for movie in wiki_movies
              if('Director' in movie or 'Directed by' in movie)
              and 'imdb_link' in movie
              and "no. of episodes" not in movie]
     # 4. Write a list comprehension to iterate through the cleaned wiki movies list
    # and call the clean_movie function on each movie.
    clean_movies = [clean_movie(movie) for movie in wiki_file]
    
    # 5. Read in the cleaned movies list from Step 4 as a DataFrame.
    wiki_movies_df = pd.DataFrame(clean_movies)
    sorted(wiki_movies_df.columns.tolist())
    
    # 6. Write a try-except block to catch errors while extracting the IMDb ID using a regular expression string and
    #  dropping any imdb_id duplicates. If there is an error, capture and print the exception.
    try:
        wiki_movies_df['imdb_id'] = wiki_movies_df['imdb_link'].str.extract(r'(tt\d{7})')
        wiki_movies_df.drop_duplicates(subset='imdb_id', inplace=True)
        
    except(IndexError):
        print('Index Error .... skipping')
        
     #  7. Write a list comprehension to keep the columns that don't have null values 
    #from the wiki_movies_df DataFrame.
    keep_columns = [column for column in wiki_movies_df.columns if wiki_movies_df[column].isnull().sum() 
                   <len(wiki_movies_df) * 0.9]
    wiki_movies_df = wiki_movies_df[keep_columns]
    
    # 8. Create a variable that will hold the non-null values from the “Box office” column.
    box_office = wiki_movies_df['Box office'].dropna()
    
    # 9. Convert the box office data created in Step 8 to string values using the lambda and 
    #join functions. Module 8.3.8
    box_office.map(lambda x: type(x) != str)
    box_office = box_office.apply(lambda x: ' '.join(x) if type(x) == list else x)
                               
    # 10. Write a regular expression to match the six elements of "form_one" of the box 
    #office data. 8.3.10
    form_one = r'\$\s*\d+\.?\d*\s*[mb]illi?on'                             
   # 11. Write a regular expression to match the three elements of "form_two" of the 
    #box office data.
    form_two = r'\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s[mb]illion)'

    # 12. Add the parse_dollars function. #8.3.10
    def parse_dollars(s):
        if type(s) != str:
            return np.nan
        if re.match(r'\$\s*\d+\.?\d*\s*milli?on', s, flags=re.I):
            s = re.sub('\$|\s|[a-zA-Z]','', s)
            value = float(s) * 10**6
            return value
        elif re.match(r'\$\s*\d+\.?\d*\s*billi?on', s, flags=re.I):
            s = re.sub('\$|\s|[a-zA-Z]','', s)
            value = float(s) * 10**9
            return value
        elif re.match(r'\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s[mb]illion)', s, flags=re.I):
            s = re.sub('\$|,','', s)
            value = float(s)
            return value
        else:
            return np.nan
    # 13. Clean the box office column in the wiki_movies_df DataFrame.
    wiki_movies_df['box_office'] = box_office.str.extract(f'({form_one}|{form_two})', 
                                    flags=re.I)[0].apply(parse_dollars)
    wiki_movies_df.drop('Box office', axis=1, inplace=True)
    # 14. Clean the budget column in the wiki_movies_df DataFrame.
    budget = wiki_movies_df['Budget'].dropna()
    budget = budget.map(lambda x: ' '.join(x) if type(x) == list else x)
    budget = budget.str.replace(r'\$.*[-—–](?![a-z])', '$', regex=True)
    budget = budget.str.replace(r'\[\d+\]\s*', '')
    wiki_movies_df['budget'] = budget.str.extract(f'({form_one}|{form_two})', 
                                flags=re.I)[0].apply(parse_dollars)
    wiki_movies_df.drop('Budget', axis=1, inplace=True)

    # 15. Clean the release date column in the wiki_movies_df DataFrame.
    release_date = wiki_movies_df['Release date'].dropna().apply(lambda x: ' '.join(x) 
                                if type(x) == list else x)
    date_form_one = r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s[123]?\d,\s\d{4}'
    date_form_two = r'\d{4}.[01]\d.[0123]\d'
    date_form_three = r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s\d{4}'
    date_form_four = r'\d{4}'
    release_date.str.extract(f'({date_form_one}|{date_form_two}|{date_form_three}|{date_form_four})', flags=re.I)
    wiki_movies_df['release_date'] = pd.to_datetime(release_date.str.extract(f'({date_form_one}|{date_form_two}|{date_form_three}|{date_form_four})')[0], infer_datetime_format=True)
    
    
   # 16. Clean the running time column in the wiki_movies_df DataFrame.
    running_time = wiki_movies_df['Running time'].dropna().apply(lambda x: ' '.join(x) if type(x) == list else x)
    running_time[running_time.str.contains(r'^\d*\s*minutes$', flags=re.I, na=False) != True]
    running_time[running_time.str.contains(r'^\d*\s*m', flags=re.I, na=False) != True]
    running_time_extract = running_time.str.extract(r'(\d+)\s*ho?u?r?s?\s*(\d*)|(\d+)\s*m')
    running_time_extract = running_time_extract.apply(lambda col: pd.to_numeric(col, errors='coerce')).fillna(0)
    wiki_movies_df['running_time'] = running_time_extract.apply(lambda row: row[0]*60 + row[1] if row[2] == 0 else row[2], axis=1)
    wiki_movies_df.drop('Running time', axis=1, inplace=True)
    
    
   # 2. Clean the Kaggle metadata.
    kaggle_file[~ kaggle_file['adult'].isin(['True', 'False'])]
    kaggle_file = kaggle_file[kaggle_file['adult'] == 'False'].drop('adult',axis='columns')

    kaggle_file['video']  == 'True'
    kaggle_file['video'] = kaggle_file['video'] =='True'
   
    kaggle_file['budget'] = kaggle_file['budget'].astype(int)
    kaggle_file['id'] = pd.to_numeric(kaggle_file['id'], errors='raise')
    kaggle_file['popularity'] = pd.to_numeric(kaggle_file['popularity'], errors='raise')
    kaggle_file['release_date'] = pd.to_datetime(kaggle_file['release_date'])
    
    
    # 3. Merged the two DataFrames into the movies DataFrame.
    
    movies_df = pd.merge(wiki_movies_df, kaggle_file, on='imdb_id', suffixes=['_wiki','_kaggle'])

    # 4. Drop unnecessary columns from the merged DataFrame.
    movies_df.drop(columns=['title_wiki','release_date_wiki','Language','Production company(s)'], inplace=True)

    # 5. Add in the function to fill in the missing Kaggle data.
    def fill_missing_kaggle_data(df, kaggle_column, wiki_column):
        df[kaggle_column] = df.apply(
        lambda row: row[wiki_column] if row[kaggle_column] == 0 else row[kaggle_column]
        , axis=1)
        df.drop(columns=wiki_column, inplace=True)
    
  

    # 6. Call the function in Step 5 with the DataFrame and columns as the arguments.
    fill_missing_kaggle_data(movies_df, 'runtime', 'running_time')
    fill_missing_kaggle_data(movies_df, 'budget_kaggle', 'budget_wiki')
    fill_missing_kaggle_data(movies_df, 'revenue', 'box_office')
   

    # 7. Filter the movies DataFrame for specific columns.
    movies_df = movies_df.loc[:, ['imdb_id','id','title_kaggle','original_title','tagline','belongs_to_collection','url','imdb_link',
                       'runtime','budget_kaggle','revenue','release_date_kaggle','popularity','vote_average','vote_count',
                       'genres','original_language','overview','spoken_languages','Country',
                       'production_companies','production_countries','Distributor',
                       'Producer(s)','Director','Starring','Cinematography','Editor(s)','Writer(s)','Composer(s)','Based on'
                      ]]

    # 8. Rename the columns in the movies DataFrame.
    movies_df.rename({'id':'kaggle_id', 'title_kaggle':'title', 'url':'wikipedia_url',
                  'budget_kaggle':'budget', 'release_date_kaggle':'release_date',  
                  'Country':'country', 'Distributor':'distributor',                
                  'Producer(s)':'producers',  'Director':'director',       
                  'Starring':'starring',  'Cinematography':'cinematography',    
                  'Editor(s)':'editors', 'Writer(s)':'writers', 
                  'Composer(s)':'composers', 'Based on':'based_on'
                 }, axis='columns', inplace=True)

    # 9. Transform and merge the ratings DataFrame.
    #9a transform rratings timestamp
    ratings_file['timestamp'] = pd.to_datetime(ratings_file['timestamp'], unit='s')
    rating_counts = ratings_file.groupby(['movieId','rating'], as_index=False).count() \
                .rename({'userId':'count'}, axis=1)\
                .pivot(index='movieId',columns='rating', values='count')
    rating_counts.columns = ['rating_' + str(col) for col in rating_counts.columns]
    
    #9b merge ratings with moveis_df
    movies_with_ratings_df = pd.merge(movies_df, rating_counts, left_on='kaggle_id', right_index=True, how='left')

    
    
    return wiki_movies_df,kaggle_file, ratings_file, movies_df, movies_with_ratings_df


In [96]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7037 entries, 0 to 7079
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   url                    7037 non-null   object        
 1   year                   7037 non-null   int64         
 2   imdb_link              7037 non-null   object        
 3   title                  7036 non-null   object        
 4   Based on               2182 non-null   object        
 5   Starring               6852 non-null   object        
 6   Cinematography         6344 non-null   object        
 7   Release date           7005 non-null   object        
 8   Country                6801 non-null   object        
 9   Language               6789 non-null   object        
 10  Director               7037 non-null   object        
 11  Distributor            6677 non-null   object        
 12  Editor(s)              6489 non-null   object        
 13  Com

In [100]:
# 11. Set the three variables equal to the function created in D1.
wiki_movies_df, movies_df, movies_with_ratings_df = ETL()

/Users/laurenerminina/opt/anaconda3/envs/PythonData/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:84: FutureWarning: The default value of regex will change from True to False in a future version.


ValueError: too many values to unpack (expected 3)

In [90]:
ratings_file.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6054 entries, 0 to 6053
Data columns (total 31 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   imdb_id                6054 non-null   object        
 1   kaggle_id              6054 non-null   int64         
 2   title                  6054 non-null   object        
 3   original_title         6054 non-null   object        
 4   tagline                4919 non-null   object        
 5   belongs_to_collection  1029 non-null   object        
 6   wikipedia_url          6054 non-null   object        
 7   imdb_link              6054 non-null   object        
 8   runtime                6053 non-null   float64       
 9   budget                 4613 non-null   float64       
 10  revenue                5172 non-null   float64       
 11  release_date           6054 non-null   datetime64[ns]
 12  popularity             6054 non-null   float64       
 13  vot

In [91]:
kaggle_file.dtypes

userId                int64
movieId               int64
rating              float64
timestamp    datetime64[ns]
dtype: object

In [ ]:
# 12. Set the DataFrames from the return statement equal to the file names in Step 11. 
wiki_movies_df = wiki_file
movies_with_ratings_df = kaggle_file
movies_df = ratings_file

In [ ]:
# 13. Check the wiki_movies_df DataFrame. 
wiki_movies_df.head()

In [ ]:
# 14. Check the movies_with_ratings_df DataFrame.
movies_with_ratings_df.head()

In [ ]:
# 15. Check the movies_df DataFrame. 
movies_df.head()